In [3]:
from pymongo import MongoClient

from RegAPI import Reg_API
from APIScraper import CommentParser

# Setup basic MongoDB collection
client = MongoClient()
db = client.regulationsgov
comments_collection = db.comments

In [9]:
api = Reg_API(20)
comment_link = api.endpoint("/comments").get()["data"][0]["links"]["self"]
comment_details = api.url(comment_link).get()

In [16]:
api.endpoint("/comments").get(get_json=False).headers

{'Date': 'Tue, 25 Feb 2025 01:31:55 GMT', 'Content-Type': 'application/vnd.api+json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Age': '1', 'Strict-Transport-Security': 'max-age=31536000; preload', 'Via': 'https/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Api-Umbrella-Request-Id': 'cl7atm8sstllsd820jog', 'X-Cache': 'MISS', 'X-Content-Type-Options': 'nosniff', 'X-Ratelimit-Limit': '1000', 'X-Ratelimit-Remaining': '999', 'X-Vcap-Request-Id': '6302fabb-7ef7-45f1-6572-7b78f6c1387b', 'X-Xss-Protection': '1; mode=block', 'X-Frame-Options': 'DENY'}

In [15]:
comment_details['data']["attributes"]

{'commentOn': '090000648293ead2',
 'commentOnDocumentId': 'EPA-R10-OW-2017-0369-0001',
 'duplicateComments': 1,
 'address1': None,
 'address2': None,
 'agencyId': 'EPA',
 'city': None,
 'category': None,
 'comment': 'See attached',
 'country': None,
 'displayProperties': [{'name': 'pageCount',
   'label': 'Page Count',
   'tooltip': 'Number of pages In the content file'}],
 'docAbstract': None,
 'docketId': 'EPA-R10-OW-2017-0369',
 'documentType': 'Public Submission',
 'email': None,
 'fax': None,
 'field1': None,
 'field2': None,
 'fileFormats': None,
 'firstName': None,
 'govAgency': None,
 'govAgencyType': None,
 'objectId': '0900006482ba59a6',
 'lastName': None,
 'legacyId': None,
 'modifyDate': '2017-10-12T01:52:49Z',
 'organization': None,
 'originalDocumentId': None,
 'pageCount': 1,
 'phone': None,
 'postedDate': '2017-10-11T04:00:00Z',
 'postmarkDate': '2017-06-11T04:00:00Z',
 'reasonWithdrawn': None,
 'receiveDate': '2017-06-11T04:00:00Z',
 'restrictReason': None,
 'restrictR

In [ ]:
page_size = 20
api = Reg_API(page_size)
parser = CommentParser(api)

pageNum = 1
while True: 
    comments = api.endpoint("/comments").search("climate").page(pageNum).get()
    
    for i, comment in enumerate(comments["data"]):
        comment_data = parser.get_comment_data(comment)
        print(f"(pg {pageNum}) {i+1}/{page_size}: ", comment_data['_id'], end="")
        print(" "*50, end="\r") # Clear line

        ## Uncomment to store in MongoDB
        #comments_collection.insert_one(comment_data) 

    if pageNum > 1: ### Comment out/delete to run for ALL pages
        break       # - Currently stops after 2nd page
    
    if comments["meta"]["hasNextPage"] == False:
        break
    else:
        pageNum += 1

In [4]:
import os
os.getcwd()

'/home/emmaf/School/Courses/CMSE-495/TwoSix_Spring25/data_collection/API'